In [3]:
import springernature_api_client.meta as meta
from dotenv import get_key
import json

meta_client = meta.MetaAPI(api_key=get_key(".env", "SPRINGER_API_KEY"))

# Get possible abstracts

In [ ]:
s=0
m=4575
res = []

for i in range(s, int(m / 25)):
    response = meta_client.search(q='human-ai interaction', p=25, s=(i * 25)+1, fetch_all=False, is_premium=False)
    res.append(response['records'])
    
flatten = [x for xs in res for x in xs]
with open('springer-raw.json', 'w') as f:
    f.write(json.dumps(flatten))

# Match abstract with paper doi

In [ ]:
abstract = ""
springer_dois = ""

with open('data/doi/springer.txt', 'r') as f:
    springer_dois = set([ "doi:" + i.split('/article/')[1] for i in f.read().split("\n") ])

with open('data/abstract/springer-raw.json', 'r') as f:
    abstract = json.loads(f.read())

In [6]:
springer_papers = []

for i in abstract:
    if(i['identifier'] in springer_dois):
        springer_papers.append({
            "paperId": i['identifier'].split("doi:")[1],
            "title": i['title'],
            "abstract": i['abstract'],
            "publicationDate": i['publicationDate']
        })

In [8]:
import springernature_api_client.meta as meta
from dotenv import get_key
meta_client = meta.MetaAPI(api_key=get_key(".env", "SPRINGER_API_KEY"))

missings = springer_dois - set(["doi:" + i['paperId'] for i in springer_papers])
for i in [*missings]:
    paper = meta_client.search(q=i)['records'][0]
    springer_papers.append({
            "paperId": paper['identifier'].split("doi:")[1],
            "title": paper['title'],
            "abstract": paper['abstract'],
            "publicationDate": paper['publicationDate']
        })

2025-04-08 01:48:45,998 - INFO - Making request to: https://api.springernature.com/meta/v2/json with params: {'q': 'doi:10.1007/s00146-023-01832-9', 'p': 10, 's': 1, 'api_key': 'b007d43ac03eb7d2fa79f1b448261ec8'}


Fetching meta: query='doi:10.1007/s00146-023-01832-9', page=10, start=1


2025-04-08 01:48:47,191 - INFO - Making request to: https://api.springernature.com/meta/v2/json with params: {'q': 'doi:10.1007/s00146-017-0732-5', 'p': 10, 's': 1, 'api_key': 'b007d43ac03eb7d2fa79f1b448261ec8'}


Fetching meta: query='doi:10.1007/s00146-017-0732-5', page=10, start=1


In [ ]:
import functions.fetch as fetch
fetch.save(json.dumps(springer_papers), "data/abstract/springer.json")